In [1]:
import pandas as pd
import os
import datetime as dt
import spmfunctions.spmfunctions as spm
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
intx='US-20-26&Middleton'#'us95&30'#'ACHD_Chinden-SH16'#'ACHD_Eagle-StLuke'#'US-20-26&Middleton'#'SH-52-Main'#'SH-44-Hartley'#'ACHD_Eagle-StLuke'
f_pkl='df_raw.pk'#'df_raw_2401.pk'
sfx=''

path_pkl = '.\\{}\\df_raw'.format(intx)
df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))

spm.plot_term(df_raw,'./{}/Plotting/term.html'.format(intx),intx)

In [4]:
#intx='US-20-26&Middleton'#'SH-52-Main'#'SH-44-Hartley'#'ACHD_Eagle-Franklin'#'ACHD_Eagle-StLuke'#
#f_pkl='df_raw2405.pk'#'df_raw_2312.pk'#
#sfx=''

path_pkl = '.\\{}\\df_raw'.format(intx)
df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))

both = []#[4]#[4,8]

dates = ['1-15-2025', '1-29-2025']#,'4-26-2024','4-27-2024','4-28-2024','4-29-2024','4-30-2024']#,'1-30-2024']#['1-13-2024', '1-14-2024','1-15-2024','1-16-2024','1-17-2024','1-18-2024']#,'11-30-2023']#,'12-1-2023','12-2-2023',
      #'12-3-2023','12-4-2023','12-5-2023']


for dt_s in dates:
    
    df_comb=spm.comb_gyr_det(spm.filter_date(df_raw,dt_s,dt_s,add_day=True))
    save_path='./{}/Plotting'.format(intx)
    sfx=dt_s
    ring_dict=spm.ring_dfs(df_comb,both=both)
    spm.plot_coord(ring_dict,df_comb,'./{}/Configuration'.format(intx),path_out=save_path,sfx=sfx)
    #plot_coord_plotly(ring_dict,df_comb,'./{}/Configuration'.format(intx),path_out=save_path,sfx=sfx+'Duration')
    
spm.plot_term(df_raw,'./{}/Plotting/term.html'.format(intx),intx)

In [12]:
spm.plot_term(df_raw,'./{}/Plotting/term.html'.format(intx),intx)

In [4]:
df_raw.loc[df_raw.Code.isin([82,81]).index,'Next']=df_raw[df_raw.Code.isin([82,81])].groupby('ID',as_index=False)['Code'].shift(-1)#.Code
df_raw.loc[df_raw.Code.isin([82,81]).index,'TS_end']=df_raw[df_raw.Code.isin([82,81])].groupby('ID',as_index=False)['TS_start'].shift(-1)#.TS_start
df_raw.loc[:,'Duration']=(df_raw.TS_end-df_raw.TS_start).dt.total_seconds()

In [25]:
df_81=df_raw[df_raw.Code==81]
df_81=spm.filter_time(df_81,'6:00','20:00','TS_start')
df_81[df_81.ID.isin([8,19])].sort_values('Duration', ascending=False).to_csv('descending_det_actuations.csv',index=False)

In [7]:
df_plt=df_raw[df_raw.Code==1].groupby(['Cycle_start','ID']).Code.count().unstack('ID').fillna(0).loc[:,4].rolling(10).mean()
plot_ts(df_plt, 'line', 'count of ph4 skips.html', title='Ph 4 Served', y_lab='Pct of cycles served')

In [8]:
df_arr=df_raw.loc[df_raw.Code.isin([81,82]),['TS_start','Code','ID']].dropna()
#df_arr.loc[:,'Code']=df_arr.Code.apply(str)
#df_arr.loc[df_arr.Code==81, 'Symbol']='arrow-left'
#df_arr.loc[df_arr.Code==82, 'Symbol']='arrow-right'

for d in ['9-11-2024','9-12-2024','9-13-2024']:
    plot_multiple_ts(spm.filter_date(df_arr,d,d,'TS_start'), 'TS_start','ID','scatter', f'dets_{d}.html')

In [5]:
# NOT a fully functioning function. Works as used in previous cell for plotting scatter, but 'line' option references un-initiated variable "r"

def plot_multiple_ts(df, xcol, ycol, ty, save_path, title = '', y_lab = ''):
    
    fig = make_subplots(rows=1, cols=1, subplot_titles=[title])
    
    if ty=='line':
        fig.add_trace(px.line(x=[r.TS_start,r.TS_end], y=[r.ID,r.ID]).data[0])
    elif ty=='scatter':
        fig.add_trace(px.scatter(x=df.loc[:,xcol], y=df.loc[:,ycol], color=df.Code).data[0])

    fig.update_layout(xaxis=dict(title='Time', type="date"),
                      yaxis=dict(title=y_lab))

    fig.update_layout(showlegend=False)
    #fig.update_layout(title_text=f"{intx} Phase Termination")
    fig.layout.xaxis.autorange = True
    fig.layout.yaxis.autorange = True
    
    fig.write_html(save_path)    

In [6]:
def plot_ts(srs, ty, save_path, title='', y_lab=''):

    
    fig = make_subplots(rows=1, cols=1, subplot_titles=[title])
    
    if ty=='line':
        fig.add_trace(go.Scatter(x=srs.index, y=srs, mode='lines', line=dict(color='black')))
    elif ty=='scatter':
        fig.add_trace(go.Scatter(x=srs.index, y=srs, mode='markers', marker=dict(color='green',symbol='circle')))

    fig.update_layout(xaxis=dict(title='Time', type="date"),
                      yaxis=dict(title=y_lab))

    fig.update_layout(showlegend=False)
    #fig.update_layout(title_text=f"{intx} Phase Termination")

    fig.write_html(save_path)

In [2]:
intersections=['Chinden','Colchester','Fairview','Franklin',
               'HobbleCreek','I84EB','I84WB','IslandWoods',
               'McMillan','Pine','Riverside','RiverValley',
               'StLuke','Ustick','Wainwright']
intersections=['StLuke']
dates=['4-1-2023','12-28-2023','5-25-2023','10-10-2023',
      '4-4-2023','4-5-2023','9-16-2023','7-23-2023']

for intx in intersections:
    print('Starting {} {}'.format(intx, dt.datetime.now()))
    for dt_s in dates:
        mm='{:02d}'.format(int(dt_s.split('-')[0]))
        yy='{}'.format(int(dt_s.split('-')[2][-2:]))
        f_pkl='df_raw_{}{}.pk'.format(yy,mm)
        path_pkl = '.\\ACHD_Eagle-{}/df_raw'.format(intx)
        df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))
        df_comb=spm.comb_gyr_det(spm.filter_date(df_raw,dt_s,dt_s))
        save_path='./ACHD_Eagle-{}/Plotting'.format(intx)
        sfx=dt_s+'_EVPE'
        ring_dict=spm.ring_dfs(df_comb)
        spm.plot_coord(ring_dict,df_comb,'./ACHD_Eagle-{}/Configuration'.format(intx),path_out=save_path,sfx=sfx)
    print('Finished {} {}'.format(intx, dt.datetime.now()))

Starting StLuke 2024-01-30 10:52:52.413753
Finished StLuke 2024-01-30 10:55:20.875067
